In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [4]:
def make_soup(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'lxml')
    return soup

In [29]:
class pbp_drive():
    
    def parse_drive(self,drive):
        
        # Try to read drive header
        header = drive.find("div",{"class":"accordion-header"})
        self.is_half = False
        if header == None: # Then we've got the end of a half or something
            self.is_half = True
#            print("found end of half, etc.")
            text = drive.find("span",{"class":"post-play"}).contents
            df = pd.DataFrame([[[],text]], columns=['downdist','detail'])
            return df
        
        # Grab information from the drive header
        possessor_logo = drive.find("span",{"class":"home-logo"}).contents[0]
        s = "nfl/500/"
        e = ".png"
        # Cut off pieces of url before and after home team
        self.offense = (str(possessor_logo).split(s))[1].split(e)[0].upper()
        # Get result of the drive
        self.result = header.find("span",{"class":"headline"}).contents
        # Get info about home/away score
        home_info = header.find("span",{"class":"home"}).contents
        self.home_team = home_info[0].contents[0]
        self.home_score_after = home_info[1].contents[0]
        away_info = header.find("span",{"class":"away"}).contents
        self.away_team = away_info[0].contents[0]
        self.away_score_after = away_info[1].contents[0]
        # Get drive summary
        self.drive_detail = header.find("span",{"class":"drive-details"}).contents
#        print(self.drive_detail)
        self.num_plays = self.drive_detail[0].split()[0]
        self.num_yards = self.drive_detail[0].split()[2]
        self.time_of_poss = self.drive_detail[0].split()[4]
#        print(self.result)
#        print([self.home_team,self.home_score_after,self.away_team,self.away_score_after])
    
        # Make a dataframe for the drive
        # Grab info about individual plays from this drive
        playlist = []
        plays = drive.find_all("li")
        for p in plays:
            try:
                downdist = p.h3.contents
                detail = p.span.contents[0].replace("\n","").replace("\t","")
                playlist.append([downdist,detail])
#                print([downdist,detail])
            except:
                pass
            
        df = pd.DataFrame(playlist, columns=['downdist','detail'])
        return df


In [33]:
# Putting all of the pieces together
def get_game_df(url):
    
    # Make a soup object with html
    soup = make_soup(url)
    
    # Find article with play-by-play table
    article = soup.find("article", {"class":"sub-module play-by-play"})
    # Article is constructed like accordion, with items corresponding
    # to individual drives
    accordion = article.find("ul", {"class":"css-accordion"})
    drives = accordion.find_all("li", {"class":"accordion-item"})
    
    # Now parse each of the drives into a dataFrame
    drivelist = []
    for i, drive in enumerate(drives):
        # Initialize drive object, then parse 
        d = pbp_drive()
        d.df = d.parse_drive(drive)
        d.drive_num = i
    
        if i == 0:
            d.home_score = 0
            d.away_score = 0
        else:
            d.home_score = drivelist[-1].home_score_after
            d.away_score = drivelist[-1].away_score_after
    
        # If the drive isn't a special section marking the end of half/game
        # Then add drive's dataFrame to the drive list
        if not d.is_half:
            d.df['home'] = d.home_team
            d.df['away'] = d.away_team
            d.df['possession'] = d.offense
            d.df['home_score'] = d.home_score
            d.df['away_score'] = d.away_score
            d.df['drive_num'] = d.drive_num

            #print(d.df)
            drivelist.append(d)

    # Make a dataFrame for individual drives
    drive_dicts = [{'drive':dd.drive_num,
                    'offense':dd.offense,
                    'plays':dd.num_plays,
                    'yds_gained':dd.num_yards,
                    'time':dd.time_of_poss,
                    'result':dd.result[0],
                    'home':dd.home_team,
                    'away':dd.away_team,
                    'home_score_after':dd.home_score,
                    'away_score_after':dd.away_score }
                   for dd in drivelist ]
    drives_df = pd.DataFrame(drive_dicts)
            
    pbp_df = pd.concat([d.df for d in drivelist])
    return pbp_df, drives_df

In [34]:
game, drives = get_game_df("http://www.espn.com/nfl/playbyplay?gameId=400951568")
game.head(15)
drives.head(10)

,away,away_score_after,drive,home,home_score_after,offense,plays,result,time,yds_gained
0,SF,0,0,LAR,0,SF,1,Interception,0:08,0
1,SF,0,1,LAR,0,LAR,1,Touchdown,0:04,3
2,SF,0,2,LAR,7,SF,14,Touchdown,6:31,81
3,SF,7,3,LAR,7,LAR,8,Touchdown,3:45,75
4,SF,7,4,LAR,14,SF,5,Fumble,1:27,42
5,SF,7,5,LAR,14,LAR,7,Field Goal,3:10,38
6,SF,7,6,LAR,17,SF,6,Punt,3:20,32
7,SF,7,7,LAR,17,SF,4,Field Goal,1:58,-6
8,SF,10,8,LAR,17,LAR,3,Punt,1:07,-2
9,SF,10,9,LAR,17,SF,8,Field Goal,4:44,47


In [35]:
# Function to get gameIds for a particular year/week
results = {}
def get_gameId(year,week):
    # Make a soup object for the appropriate page
    url = "http://www.espn.com/nfl/schedule/_/week/{0}/year/{1}".format(week,year)
    soup = make_soup(url)
    sched_page = soup.find("section",{"id":"main-container"})
    
    # Make a list for gameIds
    gameids = []
    for link in sched_page.find_all('a'):
        if "gameId" in link.get('href'):
            # Extract last bit of url listed
            s = "gameId="
            this_game = link.get('href').split(s)[1]
            gameids.append(this_game)
            # And add text displayed to a dictionary
            results[this_game] = link.contents[0]
    
    return gameids, results

In [36]:
gameids, results = get_gameId(2017,1)
print(gameids)
print(results)

['400951566', '400951567', '400951570', '400951572', '400951574', '400951576', '400951584', '400951592', '400951580', '400951597', '400951601', '400951605', '400951608', '400951581', '400951612', '400951615']
{'400951566': 'KC 42, NE 27', '400951567': 'BUF 21, NYJ 12', '400951570': 'ATL 23, CHI 17', '400951572': 'BAL 20, CIN 0', '400951574': 'PIT 21, CLE 18', '400951576': 'DET 35, ARI 23', '400951584': 'OAK 26, TEN 16', '400951592': 'PHI 30, WSH 17', '400951580': 'JAX 29, HOU 7', '400951597': 'LAR 46, IND 9', '400951601': 'GB 17, SEA 9', '400951605': 'CAR 23, SF 3', '400951608': 'DAL 19, NYG 3', '400951581': 'Postponed', '400951612': 'MIN 29, NO 19', '400951615': 'DEN 24, LAC 21'}


In [47]:
# Loop over desired weeks/years to get gameIds that can be used to look up play-by-play for each of the games
import time
gameids = []
gameresults = {}
gameyear = {}
gameweek = {}
for year in range(2009,2017):
    for week in range(1,18):
        print("Looking up gameIds for {0} week {1}".format(year,week))
        ids, results = get_gameId(year,week)
        gameids.append(ids)
        
        # Add entry in dictionaries for each gameId
        for i in ids:
            gameyear[i] = year
            gameweek[i] = week
            gameresults[i] = results[i]
        
        # Sleep so we don't get blocked
        time.sleep(0.5)
        
# Now flatten gameids, which is a list of weekly lists
ids = [i for sublist in gameids for i in sublist]
print(gameresults)

Looking up gameIds for 2009 week 1
Looking up gameIds for 2009 week 2
Looking up gameIds for 2009 week 3
Looking up gameIds for 2009 week 4
Looking up gameIds for 2009 week 5
Looking up gameIds for 2009 week 6
Looking up gameIds for 2009 week 7
Looking up gameIds for 2009 week 8
Looking up gameIds for 2009 week 9
Looking up gameIds for 2009 week 10
Looking up gameIds for 2009 week 11
Looking up gameIds for 2009 week 12
Looking up gameIds for 2009 week 13
Looking up gameIds for 2009 week 14
Looking up gameIds for 2009 week 15
Looking up gameIds for 2009 week 16
Looking up gameIds for 2009 week 17
Looking up gameIds for 2010 week 1
Looking up gameIds for 2010 week 2
Looking up gameIds for 2010 week 3
Looking up gameIds for 2010 week 4
Looking up gameIds for 2010 week 5
Looking up gameIds for 2010 week 6
Looking up gameIds for 2010 week 7
Looking up gameIds for 2010 week 8
Looking up gameIds for 2010 week 9
Looking up gameIds for 2010 week 10
Looking up gameIds for 2010 week 11
Looking up

In [48]:
# Make dataframe for game-specific information using our dictionaries for year and week
data = [ {'gameId':i,
          'season':gameyear[i],
          'week':gameweek[i],
          'result':gameresults[i]}
        for i in ids]
gamedata_df = pd.DataFrame(data)
# Set the gameId as the unique identifier for each row
gamedata_df.set_index('gameId', inplace=True)
gamedata_df.sample(10)

,result,season,week
gameId,,,
400554322,"NE 34, DET 9",2014,12
400791605,"TB 23, ATL 20 (OT)",2015,8
321122008,"HOU 34, DET 31 (OT)",2012,12
321223007,"DEN 34, CLE 12",2012,16
301107029,"NO 34, CAR 3",2010,9
330915034,"HOU 30, TEN 24 (OT)",2013,2
320101019,"NYG 31, DAL 14",2011,17
290920003,"CHI 17, PIT 14",2009,2
331103006,"DAL 27, MIN 23",2013,9


In [49]:
# Create list of individual game dataFrames
pbp_list = []
drivelevel_list = []
game_home = {}
game_away = {}

# Now loop over gameIds to scrape individual game play-by-play
for i in ids:
    
    # Check year of game. Only search for pbp of games from 2004 or later.
    if gameyear[i] >= 2004:
        
        try:
    
            print(i)
            # Make whole url for play-by-play
            url = "http://www.espn.com/nfl/playbyplay?gameId="+i
    
            # Call function to scrape and parse info into a dataFrame
            pbp_df, drives_df = get_game_df(url)
    
            # Add column to dataframe for gameId
            pbp_df['gameId'] = i
            drives_df['gameId'] = i
    
            # Extract home/away from game df
            game_home[i] = pbp_df['home'].values[0]
            game_away[i] = pbp_df['away'].values[0]

            pbp_list.append(pbp_df)
            drivelevel_list.append(drives_df)
            
        except:
            print("Failed to scrape gameId "+i)
            pass
        
        time.sleep(0.5)

    
# Put individual game dataFrames together into one big dataFrame
allplays_df = pd.concat(pbp_list)
alldrives_df = pd.concat(drivelevel_list)

290910023
290913001
290913004
290913005
290913011
290913018
290913027
290913029
290913033
290913034
290913019
290913022
290913026
290913009
290914017
290914013
290920001
290920008
290920009
290920010
290920012
290920020
290920021
290920028
290920030
290920002
290920025
290920003
290920007
290920024
290920006
290921015
290927008
290927014
290927016
290927017
290927020
290927021
290927027
290927033
290927034
290927002
290927026
290927004
290927013
290927024
290927022
290928006
291004003
291004005
291004011
291004012
291004017
291004028
291004030
291004034
291004015
291004018
291004007
291004025
291004023
291005016
291011002
291011008
291011012
291011014
291011019
291011021
291011029
291011033
291011025
291011007
291011022
291011026
291011010
291012015
291018004
291018009
291018016
291018018
291018023
291018027
291018028
291018030
291018013
291018026
291018017
291018020
291018001
291019024
291025005
291025012
291025014
291025023
291025027
291025034
291025013
291025029
291025004
291025006


320927033
320930001
320930002
320930008
320930012
320930014
320930020
320930034
320930007
320930022
320930030
320930009
320930027
320930021
321001006
321004014
321007004
321007011
321007012
321007019
321007023
321007028
321007029
321007030
321007016
321007017
321007025
321007018
321008020
321011010
321014001
321014005
321014015
Failed to scrape gameId 321014015
321014020
321014021
321014027
321014033
321014022
321014026
321014025
321014028
321014034
321015024
321018025
321021002
321021011
321021014
321021016
321021019
321021027
321021029
321021034
321021013
321021017
321021004
321022003
321025016
321028003
321028005
321028008
321028009
321028010
321028014
321028020
321028021
321028023
321028012
321028006
321028007
321029022
321101024
321104004
321104005
321104009
321104010
321104011
321104028
321104030
Failed to scrape gameId 321104030
321104034
321104013
321104026
321104019
321104001
321105018
321108030
321111004
321111015
321111016
321111017
321111018
321111027
321111029
321111033
32

400791584
400791588
400791593
400791627
400791633
400791629
400791639
400791625
400791598
400791621
400791637
400791663
400791667
400791670
400791674
400791676
400791503
400761515
400791523
400791517
400791520
400791527
400791529
400791533
400791536
400791511
400791540
400791564
400791567
400791574
400791601
400761516
400791605
400791642
400791609
400791613
400791618
400791649
400791646
400791653
400791657
400791681
400791684
400791690
400791698
400791725
400791727
400791721
400791722
400791728
400791730
400791700
400791732
400791731
400791733
400791734
400791735
400791486
400791490
400791499
400791543
400791552
400791556
400791494
400791549
400791581
400791586
400791595
400791591
400791599
400791622
400791632
400791640
400791673
400791636
400791703
400791668
400791707
400791679
400791665
400791715
400791677
400791713
400791710
400791717
400791507
400791509
400791510
400791514
400791516
400791502
400791505
400791504
400791522
400791506
400791519
400791512
400791526
400791532
400791537


In [50]:
# Add columns to gamedata_df for home and away team
gamedata_df['home'] = pd.Series(game_home)
gamedata_df['away'] = pd.Series(game_away)

print(gamedata_df[gamedata_df['season']>=2004].head(5))
allplays_df.sample(10)

                        result  season  week home away
gameId                                                
290910023  PIT 13, TEN 10 (OT)    2009     1  TEN  PIT
290913001        ATL 19, MIA 7    2009     1  MIA  ATL
290913004        DEN 12, CIN 7    2009     1  DEN  CIN
290913005       MIN 34, CLE 20    2009     1  MIN  CLE
290913011       IND 14, JAX 12    2009     1  JAX  IND


,downdist,detail,home,away,possession,home_score,away_score,drive_num,gameId
3,[2nd and 11 at BAL 43],(5:16 - 2nd) T.Brady pass short right to S.Mor...,BAL,NE,NE,7,10,6,291004017
6,[],(1:59 - 2nd) Two-Minute Warning,SEA,ARI,ARI,14,3,9,400554447
1,[2nd and 7 at TEN 7],(3:48 - 4th) C.Johnson right end pushed ob at ...,DEN,TEN,TEN,16,20,24,301003010
2,[2nd and 8 at DET 25],(11:57 - 1st) Timeout #1 by CHI at 11:57.,CHI,DET,CHI,0,0,1,321230008
5,[2nd and 9 at OAK 36],(13:46 - 4th) B.Gradkowski sacked at OAK 26 fo...,OAK,ARI,OAK,20,24,20,300926022
0,[1st and 10 at NE 14],(14:49 - 4th) T.Brady pass incomplete short m...,NE,MIA,NE,10,17,16,331215015
5,[and -1 at CIN 30],(1:38 - 2nd) S.Graham kicks 71 yards from CIN ...,KC,CIN,CIN,0,0,11,291227004
11,[4th and 8 at TEN 8],(12:44 - 4th) (Field Goal formation) J.Hanson ...,DET,TEN,DET,16,20,17,320923010
14,[3rd and 28 at PIT 49],(4:49 - 2nd) D.Dixon pass short left to S.Hol...,PIT,BAL,PIT,7,7,6,291129033
4,[3rd and 13 at IND 36],(2:18 - 1st) A.Luck pass short left to C.Flee...,DEN,IND,IND,7,3,6,331020011


In [51]:
alldrives_df.sample(10)

,away,away_score_after,drive,home,home_score_after,offense,plays,result,time,yds_gained,gameId
23,NYJ,31,24,BUF,7,NYJ,5,Touchdown,2:17,34,310102020
14,CLE,6,15,TEN,21,CLE,3,Punt,1:35,9,311002005
17,NYG,24,18,PHI,10,NYG,1,Fumble Recovery (Opponent),0:33,-2,301219019
4,SD,10,4,DEN,0,DEN,4,Field Goal,2:08,0,400874693
9,IND,17,9,JAX,3,IND,3,Punt,1:17,-2,331229011
19,NYJ,14,20,SD,27,NYJ,7,Field Goal,2:59,38,321223020
7,NYG,10,7,PHI,17,PHI,1,Touchdown,0:00,72,291213019
7,CHI,6,7,DEN,7,DEN,3,Punt,1:02,2,400791673
11,CIN,12,12,JAX,3,CIN,8,Touchdown,4:27,62,400554344
10,NE,10,10,BUF,14,NE,1,End of 1st Half,0:00,0,290914017


## Begin processing the dataFrames

In [52]:
# Get information about winning team and final scores
winner = {}
home_score = {}
away_score = {}
ot = {}
for i in list(gamedata_df.index.values):
    try:
        final = gameresults[i]
    
        # Winner should be first team listed
        winner[i] = final.split()[0]
    
        if "(OT)" in final:
            ot[i] = 1
        else:
            ot[i] = 0
        
        if game_home[i] == winner[i]:
            # Home team wins, their score is listed first
            home_score[i] = final.split()[1].rstrip(",")
            away_score[i] = final.split()[3]
        else:
            # Away team wins, their score is listed first
            home_score[i] = final.split()[3]
            away_score[i] = final.split()[1].rstrip(",")
        
        # Check for a tie
        if home_score[i] == away_score[i]:
            winner[i] = "TIE"
            
    except:
        winner[i] = "unknown"
        ot[i] = "unknown"
        home_score[i] = "unknown"
        away_score[i] = "unknown"
    
gamedata_df['winner'] = pd.Series(winner)
gamedata_df['home_score'] = pd.Series(home_score)
gamedata_df['away_score'] = pd.Series(away_score)
gamedata_df['OT'] = pd.Series(ot)

In [55]:
# Games where grabbing pbp failed have some unknown values
gamedata_df[ gamedata_df['winner'] == "unknown" ]

,result,season,week,home,away,winner,home_score,away_score,OT
gameId,,,,,,,,,
311218021,"PHI 45, NYJ 19",2011,15,NaN,NaN,unknown,unknown,unknown,unknown
321014015,"MIA 17, STL 14",2012,6,NaN,NaN,unknown,unknown,unknown,unknown
321104030,"DET 31, JAX 14",2012,9,NaN,NaN,unknown,unknown,unknown,unknown
331021019,"NYG 23, MIN 7",2013,7,NaN,NaN,unknown,unknown,unknown,unknown
331201020,"MIA 23, NYJ 3",2013,13,NaN,NaN,unknown,unknown,unknown,unknown
400554331,Postponed,2014,12,NaN,NaN,unknown,unknown,unknown,unknown
400554366,"WSH 27, PHI 24",2014,16,NaN,NaN,unknown,unknown,unknown,unknown
400554443,"NO 23, TB 20",2014,17,NaN,NaN,unknown,unknown,unknown,unknown
400874701,"WSH 27, PHI 20",2016,6,NaN,NaN,unknown,unknown,unknown,unknown


In [59]:
# Double check a game that couldn't get home/away
url = "http://www.espn.com/nfl/playbyplay?gameId="+"400554366"
g_df,d_df = get_game_df(url)

AttributeError: 'NoneType' object has no attribute 'find'

In [53]:
gamedata_df.sample(15)

,result,season,week,home,away,winner,home_score,away_score,OT
gameId,,,,,,,,,
291213001,"NO 26, ATL 23",2009,14,NO,ATL,NO,26,23,0
301226030,"WSH 20, JAX 17 (OT)",2010,16,WSH,JAX,WSH,20,17,1
400874629,"DAL 29, PHI 23 (OT)",2016,8,PHI,DAL,DAL,23,29,1
321104030,"DET 31, JAX 14",2012,9,NaN,NaN,unknown,unknown,unknown,unknown
301205027,"ATL 28, TB 24",2010,13,ATL,TB,ATL,28,24,0
331013007,"DEN 35, JAX 19",2013,6,JAX,DEN,DEN,19,35,0
320909018,"WSH 40, NO 32",2012,1,WSH,NO,WSH,40,32,0
331020023,"PIT 19, BAL 16",2013,7,BAL,PIT,PIT,16,19,0
311003027,"TB 24, IND 17",2011,4,IND,TB,TB,17,24,0


## And start working with the play-by-play data

In [61]:
# Start by saving all dataFrames to disk in case I mess anything up
gamedata_df.to_csv("../data/espn_gamedata2009-2016.csv")
allplays_df.to_csv("../data/espn_rawplays2009-2016.csv")
alldrives_df.to_csv("../data/espn_drives2009-2016.csv")

In [16]:
# Load dataframes from disk
gamedata_df = pd.read_csv("espn_2017gamedata_wk6-10.csv")
allplays_df = pd.read_csv("espn_rawplays_2017_wk6-10.csv")

In [17]:
# Take another look at what we've got so far
gamedata_df.set_index('gameId', inplace=True)
print(allplays_df.info())
allplays_df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11935 entries, 0 to 11934
Data columns (total 9 columns):
Unnamed: 0    11935 non-null int64
downdist      11935 non-null object
detail        11935 non-null object
home          11935 non-null object
away          11935 non-null object
possession    11935 non-null object
home_score    11935 non-null int64
away_score    11935 non-null int64
gameId        11935 non-null int64
dtypes: int64(4), object(5)
memory usage: 839.3+ KB
None


,Unnamed: 0,downdist,detail,home,away,possession,home_score,away_score,gameId
6074,5,['1st and 10 at NYJ 48'],(12:39 - 1st) (Shotgun) B.Powell up the middl...,ATL,NYJ,NYJ,0,0,400951721
7363,0,['1st and 10 at BUF 25'],(3:18 - 2nd) R.Groy reported in as eligible. ...,BUF,NYJ,BUF,7,10,400951743
765,3,['3rd and 8 at NE 27'],(11:28 - 2nd) (Shotgun) T.Brady pass incomple...,NE,NYJ,NE,0,14,400951766
4780,1,['1st and 10 at ATL 28'],(14:54 - 3rd) T.Coleman left guard to ATL 30 ...,ATL,NE,ATL,0,17,400951638
11636,0,[],(5:18 - 1st) B.McManus kicks 65 yards from DEN...,NE,DEN,NE,14,6,400951690


In [18]:
# Start by trying to parse down, distance, and field position

# Make lists to populate with a value for each play
down = []
dist = []
home_fieldpos = []

# Make lists out of home and away teams for comparison with field position
hometeam = allplays_df.home.values
awayteam = allplays_df.away.values

# Function to return fieldposition from the offense's point of view
def get_fieldpos(teamside,ydline,j):
    if teamside == hometeam[j]:
        # Ball is one home team's half. Location should be negative
        return -1*(50-ydline)
    elif teamside == awayteam[j]:
        return 50-ydline
    else:
        return "x"
        

for j, c in enumerate(allplays_df['downdist'].values):
    
    x = c.strip("[]'")
    # Check for an empty list. This probably means an end of quarter/half line
    if (not x) or x == None:
        down.append(0)
        dist.append(0)
        home_fieldpos.append(0)
#        print("Found empty list")
        
    else:
        
        x = [x]
        pieces = x[0].split()
#        print(pieces)
    
        # Get down
        if not pieces[0][0].isalpha():  # check is first character is alphabetic
            # Then first character is numeric. This is the down number.
            down.append(int(pieces[0][0]))
        else:
            down.append(0)
        
        # Get distance
        for i, word in enumerate(pieces):
            if word == "and":
                dist.append(pieces[i+1])  # Keep as string to preserve goal-to-go situations
        
        # Get fieldposition from the home team's perspective
        for i, word in enumerate(pieces):
            if word == "at":
                if pieces[i+1] == '50':
                    home_fieldpos.append(0)
                else:
                    teamside = pieces[i+1]
                    ydline = int(pieces[i+2])
                    
                    fieldpos = get_fieldpos(teamside,ydline,j)
                    
                    if fieldpos == "x":  # Failed to match teamside with home/away teams
                        # Change teamside in a couple cases to account for teams moving
                        # ESPN seems to always use most recent short form name in fieldposition
                        if teamside == "LAR":
                            teamside = "STL"
                        elif teamside == "LAC":
                            teamside = "SD"
                        # Try again with new teamside
                        fieldpos = get_fieldpos(teamside,ydline,j)
                    
                    if fieldpos == "x":
                        home_fieldpos.append(0)
                        print(pieces)
                        print("Failed to find side of field correctly")
                        
                    else:
                        home_fieldpos.append(fieldpos)
                        
                
#    print([down[-1], dist[-1], home_fieldpos[-1]])
allplays_df['down'] = down
allplays_df['dist'] = dist
allplays_df['home_fieldpos'] = home_fieldpos

In [19]:
allplays_df.sample(5)

,Unnamed: 0,downdist,detail,home,away,possession,home_score,away_score,gameId,down,dist,home_fieldpos
11896,5,['1st and 10 at 50'],(0:57 - 3rd) (Shotgun) J.Cutler pass incomple...,MIA,CAR,MIA,14,38,400951693,1,10,0
6385,3,['2nd and 10 at PHI 49'],(12:34 - 4th) (Shotgun) C.Wentz pass short mi...,SF,PHI,PHI,7,27,400951723,2,10,1
11540,0,['1st and 10 at SF 24'],(9:39 - 3rd) C.Hyde right tackle to SF 27 for...,NYG,SF,SF,13,17,400951688,1,10,26
3972,2,['1st and 10 at BAL 22'],(13:28 - 4th) J.McKinnon right end to BLT 18 ...,BAL,MIN,MIN,9,18,400951603,1,10,-28
11185,3,[],(2:00 - 2nd) Two-Minute Warning,HOU,LAR,HOU,7,6,400951663,0,0,0


In [20]:
# Now look to extract the time remaining (in seconds)
detail = allplays_df.detail.values

# Make lists for quarter and time_remaining
qtr = []
time_rem = []

for d in detail[:]:
#    print(d)
#    print(type(d))
    try:
        if (not d) or (d == None):
            # detail is an empty list
            qtr.append(0)
            time_rem.append("0:00")
        
        else:
            pieces = d.split()
#            print(pieces)
            if pieces[0][0] == "E":
                # Found End of Quarter/Overtime line
                qtr.append(0)
                time_rem.append("0:00")

            elif pieces[0][0] == "(":
                # Found beginning of standard "(1:23 - 4th)" template
                qtr.append(pieces[2][0])
                time_rem.append(pieces[0].lstrip("("))
            
            else:
                # Not sure what this is, so just be safe and go to 0:00 rem in 4th
                print(d)
                qtr.append(0)
                time_rem.append("0:00")
            
    except:
        print("Default parse failed for:")
        print(d)
        qtr.append(0)
        time_rem.append("0:00")
            
#    print(qtr[-1])
#    print(time_rem[-1])
    
allplays_df['qtr'] = qtr
allplays_df['time_rem'] = time_rem

In [21]:
allplays_df[['downdist','detail','down','dist','qtr','time_rem']].sample(25)

,downdist,detail,down,dist,qtr,time_rem
96,['2nd and 10 at CAR 35'],(13:48 - 3rd) (Shotgun) C.Newton pass deep mi...,2,10,3,13:48
237,['3rd and 1 at MIA 8'],(6:15 - 2nd) (Shotgun) T.Coleman left guard t...,3,1,2,6:15
464,['3rd and 2 at GB 35'],(2:34 - 3rd) C.Keenum pass short right to K.R...,3,2,3,2:34
9473,['1st and Goal at DET 4'],"(10:24 - 4th) (No Huddle, Shotgun) T.Montgome...",1,Goal,4,10:24
4573,['1st and 10 at DEN 43'],(11:05 - 2nd) (Shotgun) P.Rivers pass short l...,1,10,2,11:05
8670,['3rd and 9 at ARI 47'],(6:03 - 1st) (Shotgun) C.Beathard sacked at S...,3,9,1,6:03
2539,['3rd and 3 at OAK 21'],(1:10 - 2nd) (Shotgun) A.Smith pass incomplet...,3,3,2,1:10
9382,[],END QUARTER 1,0,0,0,0:00
6479,['1st and 25 at CAR 48'],(6:14 - 2nd) (Shotgun) J.Winston pass short l...,1,25,2,6:14
4106,['1st and 10 at DAL 45'],(0:00 - 2nd) (Shotgun) E.Elliott up the middl...,1,10,2,0:00


In [22]:
# Make a column for secconds remaining in the game
qtr = allplays_df.qtr.values
time_rem = allplays_df.time_rem.values

secs_rem = []

for i, tr in enumerate(time_rem):
    if qtr[i] in ["1","2","3","4"]:
        q = int(qtr[i])
    elif qtr[i] == "O":
        q = 4
    else:
        q = 0
    mins = int(tr.split(":")[0])
    secs = int(tr.split(":")[1])
    secs_rem.append( 900*(4-q) + 60*mins + secs )
    
allplays_df['secs_rem'] = secs_rem

In [23]:
allplays_df[['qtr','time_rem','secs_rem']].sample(10)

,qtr,time_rem,secs_rem
2972,4,3:31,211
10851,3,3:29,1109
5357,4,6:43,403
8849,1,8:58,3238
5471,3,12:40,1660
6868,3,7:09,1329
6674,3,13:54,1734
2119,4,0:00,0
10456,2,5:15,2115
2579,3,3:38,1118


In [24]:
allplays_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11935 entries, 0 to 11934
Data columns (total 15 columns):
Unnamed: 0       11935 non-null int64
downdist         11935 non-null object
detail           11935 non-null object
home             11935 non-null object
away             11935 non-null object
possession       11935 non-null object
home_score       11935 non-null int64
away_score       11935 non-null int64
gameId           11935 non-null int64
down             11935 non-null int64
dist             11935 non-null object
home_fieldpos    11935 non-null int64
qtr              11935 non-null object
time_rem         11935 non-null object
secs_rem         11935 non-null int64
dtypes: int64(7), object(8)
memory usage: 1.4+ MB


In [25]:
# Make column for score difference
allplays_df['home_lead'] = allplays_df['home_score'] - allplays_df['away_score']

# Make column for total score
allplays_df['total_score'] = allplays_df['home_score'] + allplays_df['away_score']

In [26]:
# Make column with derived metric for adjusted lead
# Make a column for adjusted score
import math
def adjusted_lead(play):
    try:
        return play.home_lead / math.sqrt( 3600-play.secs_rem + 1 )
    except:
        return 0
    
allplays_df['adj_lead'] = allplays_df.apply(
    lambda row: adjusted_lead(row), axis=1 )

In [27]:
# Make a column for whether this play takes place in overtime
allplays_df['OT'] = [1 if " OT" in str(p) else 0 for p in allplays_df.detail.values]

In [28]:
allplays_df[allplays_df.OT == 1].sample(10)

,Unnamed: 0,downdist,detail,home,away,possession,home_score,away_score,gameId,down,dist,home_fieldpos,qtr,time_rem,secs_rem,home_lead,total_score,adj_lead,OT
11093,3,['1st and 10 at JAX 41'],(8:46 - OT) (Shotgun) B.Bortles pass short ri...,LAC,JAX,JAX,17,17,400951648,1,10,9,O,8:46,526,0,34,0.0,1
3160,1,['2nd and 3 at CLE 42'],(5:24 - OT) D.Murray right guard to CLV 42 fo...,TEN,CLE,TEN,9,9,400951582,2,3,8,O,5:24,324,0,18,0.0,1
3146,4,['3rd and 10 at CLE 25'],(9:20 - OT) (Shotgun) C.Kessler pass incomple...,TEN,CLE,CLE,9,9,400951582,3,10,25,O,9:20,560,0,18,0.0,1
11097,7,['2nd and 8 at LAC 47'],(6:37 - OT) (Shotgun) B.Bortles pass incomple...,LAC,JAX,JAX,17,17,400951648,2,8,-3,O,6:37,397,0,34,0.0,1
3156,1,['2nd and 8 at CLE 22'],(6:54 - OT) C.Kessler pass incomplete short l...,TEN,CLE,CLE,9,9,400951582,2,8,28,O,6:54,414,0,18,0.0,1
3159,0,['1st and 10 at CLE 49'],(5:57 - OT) D.Murray left guard to CLV 42 for...,TEN,CLE,TEN,9,9,400951582,1,10,1,O,5:57,357,0,18,0.0,1
1277,3,['4th and 2 at BAL 48'],(5:48 - OT) (Punt formation) S.Koch punts 45 ...,CHI,BAL,BAL,24,24,400951699,4,2,2,O,5:48,348,0,48,0.0,1
11107,3,[],(3:17 - OT) Timeout #1 by LAC at 03:17.,LAC,JAX,JAX,17,17,400951648,0,0,0,O,3:17,197,0,34,0.0,1
11104,0,['1st and 10 at LAC 17'],(5:11 - OT) L.Fournette up the middle to LAC ...,LAC,JAX,JAX,17,17,400951648,1,10,-33,O,5:11,311,0,34,0.0,1
3155,0,['1st and 10 at CLE 20'],(7:30 - OT) I.Crowell right guard to CLV 22 f...,TEN,CLE,CLE,9,9,400951582,1,10,30,O,7:30,450,0,18,0.0,1


In [29]:
# Make column for whether home team has possession
hometeam = allplays_df.home.values
awayteam = allplays_df.away.values
possession = allplays_df.possession.values

home_possession = [
    1 if hometeam[i] == p else 0 if awayteam[i] == p else "X" for i, p in enumerate(possession)
]
allplays_df['home_possession'] = home_possession

In [30]:
# Make a column for whether the hoem team wins
gamedata_df.columns

Index(['result', 'season', 'week', 'home', 'away', 'winner', 'home_score',
       'away_score', 'OT'],
      dtype='object')

In [31]:
hometeam = gamedata_df.home.values
awayteam = gamedata_df.away.values
winner = gamedata_df.winner.values

# Make new column for gamedata
home_wins = [1 if hometeam[i] == w else 0 if awayteam[i] == w else "X" for i, w in enumerate(winner)]
gamedata_df['home_win'] = home_wins

# Try pandas join
#allplays_df = allplays_df.join(gamedata_df['home_win'], on='gameId')

In [32]:
# Must be an easier way to make the home_win column but hopefully this works
joined_df = pd.merge(allplays_df, gamedata_df[['season','week','home_win']], 
                     how='left',
                     left_on='gameId',
                     right_index=True)

In [33]:
joined_df.columns

Index(['Unnamed: 0', 'downdist', 'detail', 'home', 'away', 'possession',
       'home_score', 'away_score', 'gameId', 'down', 'dist', 'home_fieldpos',
       'qtr', 'time_rem', 'secs_rem', 'home_lead', 'total_score', 'adj_lead',
       'OT', 'home_possession', 'season', 'week', 'home_win'],
      dtype='object')

In [34]:
final_cols = [
    'downdist', 'detail', 'home', 'away', 'possession',
       'home_score', 'away_score', 'gameId', 'down', 'dist', 'home_fieldpos',
       'qtr', 'time_rem', 'secs_rem', 'home_lead', 'total_score', 'adj_lead',
       'OT','home_possession','home_win','season','week'
]
print(joined_df[final_cols].describe())
joined_df[final_cols].sample(20)

         home_score    away_score        gameId          down  home_fieldpos  \
count  11935.000000  11935.000000  1.193500e+04  11935.000000   11935.000000   
mean      10.125765     11.798911  4.009517e+08      1.748052      -0.314202   
std        9.667361     10.073924  7.194157e+01      1.163426      22.436372   
min        0.000000      0.000000  4.009516e+08      0.000000     -49.000000   
25%        2.000000      3.000000  4.009516e+08      1.000000     -17.000000   
50%        7.000000     10.000000  4.009517e+08      2.000000       0.000000   
75%       16.000000     17.000000  4.009518e+08      3.000000      16.000000   
max       51.000000     52.000000  4.009518e+08      4.000000      49.000000   

           secs_rem     home_lead   total_score      adj_lead            OT  \
count  11935.000000  11935.000000  11935.000000  11935.000000  11935.000000   
mean    1750.625052     -1.673146     21.924675     -0.069224      0.005279   
std     1071.011645     11.054847     16.3

,downdist,detail,home,away,possession,home_score,away_score,gameId,down,dist,...,time_rem,secs_rem,home_lead,total_score,adj_lead,OT,home_possession,home_win,season,week
427,['1st and 40 at MIN 40'],(0:48 - 2nd) (Shotgun) J.McKinnon up the midd...,GB,MIN,MIN,10,14,400951702,1,40,...,0:48,1848,-4,24,-0.095536,0,0,0,2017,6
8791,['2nd and 8 at SF 37'],(9:52 - 4th) A.Peterson up the middle to SF 3...,ARI,SF,ARI,17,10,400951763,2,8,...,9:52,592,7,27,0.127611,0,1,1,2017,9
9090,['1st and 10 at KC 18'],(3:20 - 2nd) (Shotgun) A.Smith pass deep midd...,KC,DAL,KC,3,7,400951786,1,10,...,3:20,2000,-4,10,-0.099969,0,1,0,2017,9
1555,['1st and Goal at ARI 6'],(9:18 - 3rd) D.Martin up the middle to ARZ 4 ...,TB,ARI,TB,0,31,400951770,1,Goal,...,9:18,1458,-31,31,-0.669654,0,1,0,2017,6
10535,['1st and 10 at IND 35'],(8:39 - 4th) (Run formation) J.Conner left en...,PIT,IND,PIT,17,17,400951565,1,10,...,8:39,519,0,34,0.000000,0,1,1,2017,10
10557,['1st and 10 at IND 18'],(0:35 - 4th) (Run formation) L.Bell right end...,PIT,IND,PIT,17,17,400951565,1,10,...,0:35,35,0,34,0.000000,0,1,1,2017,10
2641,['1st and Goal at KC 2'],(0:00 - 4th) (Shotgun) D.Carr pass short left...,KC,OAK,OAK,30,24,400951571,1,Goal,...,0:00,0,6,54,0.099986,0,0,0,2017,7
10447,['2nd and 10 at PIT 49'],(8:01 - 2nd) (Shotgun) J.Brissett pass short ...,PIT,IND,IND,0,7,400951565,2,10,...,8:01,2281,-7,7,-0.192669,0,0,1,2017,10
7638,['1st and Goal at TEN 5'],(1:01 - 4th) (Shotgun) J.Flacco pass incomple...,BAL,TEN,BAL,13,23,400951761,1,Goal,...,1:01,61,-10,36,-0.168073,0,1,0,2017,9
6467,['1st and 16 at CAR 7'],(10:18 - 2nd) (Shotgun) C.Newton up the middl...,CAR,TB,CAR,7,0,400951725,1,16,...,10:18,2418,7,7,0.203519,0,1,1,2017,8


In [35]:
joined_df[final_cols].to_csv("espn_parsedplays_2017_wk6-10.csv")

## End of finished code